In [34]:
import json
import pandas as pd
import os
import sqlite3
from sqlite3 import Error
from treelib import Node, Tree
import ast
import csv

directory_with_files = '../data'
db_name = './test.db'
trees_file = './replies_trees.csv'
convs_file = './conversations.json'
airlines=[56377143, 106062176, 18332190, 22536055, 124476322, 26223583, 2182373406, 38676903, 1542862735, 253340062, 218730857, 45621423, 20626359]



In [33]:
def run_query(query, conn):
    df = pd.read_sql_query(query, conn)
    return df

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    return conn

def find_conv(user_id, check_id, db_name, conn):#redundant
    print('done')
    query = f"""
        SELECT reply_id, reply_to_status_id, reply_to_user_id, user_id, tweet_id
        FROM replies, tweets
        WHERE reply_id=tweet_id AND reply_to_status_id = {check_id}
        """
    df = run_query(query, conn)
    past_replies_dict={}
    now_reply_ids = list(df['reply_id'])
    past_replies_dict[(user_id, check_id)] = df[['user_id', 'tweet_id']].to_dict('records')
    while len(now_reply_ids) != 0:
        past_replies_dict, now_reply_ids = check_for_replies(past_replies_dict,now_reply_ids, conn)
    return past_replies_dict

def check_for_replies(past_replies_dict, new_replies_list, conn):#redundant
    now_reply_ids = []
    for a_id in new_replies_list:
        query = f"""
        SELECT reply_id, reply_to_status_id, reply_to_user_id, user_id, tweet_id
        FROM replies, tweets
        WHERE reply_id=tweet_id AND reply_to_status_id = {a_id}
        """
        return_df = run_query(query, conn)
        returned_ids = list(return_df['reply_id'])
        now_reply_ids += returned_ids
        if len(return_df)!=0:
            past_replies_dict[(return_df['reply_to_user_id'][0], return_df['reply_to_status_id'][0])] = return_df[['user_id', 'tweet_id']].to_dict('records')
        
    return past_replies_dict, now_reply_ids

def get_tweets_with_replies(db_name):
    conn = create_connection(db_name)
    query = """
    SELECT tweet_id, tweet_type, user_id, text
    FROM tweets
    WHERE tweet_id in (SELECT reply_to_status_id FROM replies) AND tweet_type = 'original'
    """
    df = run_query(query, conn)
    return df

def get_replies(db_name):
    conn = create_connection(db_name)
    query = """
    SELECT user_id, reply_id, reply_to_status_id, reply_to_user_id
    FROM replies
    INNER JOIN tweets ON replies.reply_id = tweets.tweet_id
    """
    df = run_query(query, conn)
    return df

def find_conv_2(user_id, check_id, replies):
    this_replies = replies[replies['reply_to_status_id']==check_id]
    past_replies_dict={}
    now_reply_ids = list(this_replies['reply_id'])
    past_replies_dict[(user_id, check_id)] = list(this_replies[['user_id', 'reply_id']].itertuples(index=False, name=None))
    while len(now_reply_ids) != 0:
        past_replies_dict, now_reply_ids = check_for_replies_2(past_replies_dict,now_reply_ids, replies)
    return past_replies_dict

def check_for_replies_2(past_replies_dict, new_replies_list, replies):
    now_reply_ids = []
    for a_id in new_replies_list:
        this_replies = replies[replies['reply_to_status_id']==a_id]
        returned_ids = list(this_replies['reply_id'])
        now_reply_ids += returned_ids
        if len(this_replies)!=0:
            past_replies_dict[list(this_replies[['reply_to_user_id','reply_to_status_id']].itertuples(index=False, name=None))[0]] = list(this_replies[['user_id', 'reply_id']].itertuples(index=False, name=None))
    return past_replies_dict, now_reply_ids

def build_tree(data):
    tree = Tree()
    tree.create_node(list(data.keys())[0], list(data.keys())[0])
    for key in list(data.keys()):
        for value in data[key]:
            ...
            tree.create_node(value,  value, parent = key)                                            
    return tree
                        
                
def get_all_replies_trees(tweets_with_replies,replies, db_name):

    replies_trees={}
    for n, (tweet_id, user_id) in enumerate(list(tweets_with_replies[['tweet_id', 'user_id']].itertuples(index=False, name=None))):
        if n%100==0:
            print(n/100)
        replies_trees[tweet_id] = find_conv_2(user_id, tweet_id, replies)
    return replies_trees

def get_all_replies_trees_from_csv(trees_file):#from github use this
    replies_trees={}
    with open(trees_file,'r') as data:
        for line in csv.reader(data):
            replies_trees[int(line[0])] = ast.literal_eval(", ".join(line[1:]))
    return replies_trees

def get_all_conversations(replies_trees, airlines):
    all_conversations =[]
    last=0
    dividor = len(replies_trees.keys())//100
    print("|"*100)
    for n,key in enumerate(replies_trees.keys()):
        current = n//dividor
        if current>last:
            print('|', end='')
        last=current
        built_tree = build_tree(replies_trees[key])
        all_conversations += find_conversations_in_a_tree(built_tree, airlines)
    return all_conversations

def find_conversations_in_a_tree(tree, airlines):
    conv = []
    tweets=[]
    for path in tree.paths_to_leaves():
        tweet=[]
        for n, node in enumerate(path):
            if len(conv) == 0:
                if node[0] in airlines and n!=0 and path[n-1][0] not in airlines:
                    conv.append(node[0])
                    conv.append(path[n-1][0])
                    tweet.append(path[n-1])
                    tweet.append(node)
                elif node[0] in airlines and n==0 and path[1][0] not in airlines:
                    conv.append(node[0])
                    conv.append(path[1][0])
                    tweet.append(node)
            elif len(conv) == 2:
                if node[0] in conv and node[0]!=path[n-1][0]:
                    tweet.append(node)
                else:
                    conv=[]
                    if tweet not in tweets and tweet!=[]:
                        tweets.append(tweet)
                    tweet=[]
        if tweet not in tweets and tweet!=[]:
            tweets.append(tweet)
        tweet=[]
        conv=[]
    return check_for_sublists(tweets)            

def check_for_sublists(all_convs):
    lists_to_rm=[]
    for conv in all_convs:
        for sub_conv in all_convs:
            if(all(x in conv for x in sub_conv)) and conv!=sub_conv:
                if sub_conv not in lists_to_rm:
                    lists_to_rm.append(sub_conv)
    
    for a_list in lists_to_rm:
        all_convs.remove(a_list)
    return all_convs

def get_list_of_conversations_with_text(all_conversations, db_name, airlines):
    all_conversations_list_of_dicts = []
    conn = create_connection(db_name)
    cursor = conn.cursor()
    last=0
    dividor = len(all_conversations)//100
    print("|"*100)
    for conv_id, conv in enumerate(all_conversations):
        current = conv_id//dividor
        if current>last:
            print('|', end='')
        last=current
        ids = list(set(i[0] for i in conv))
        if ids[0] in airlines:
            airline_id = ids[0]
        else:
            airline_id = ids[1]
        for tweet in conv:
            query = f'''
            SELECT text, timestamp_ms
            FROM tweets
            WHERE tweet_id = {tweet[1]}
            '''
            cursor.execute(query)
            result = cursor.fetchone()
            text = result[0]
            timestamp = result[1]
            user_id = tweet[0]
            tweet_id = tweet[1]
            all_conversations_list_of_dicts.append({'conv_id':conv_id, 'user_id':user_id, 'tweet_id':tweet_id, 'text':text, 'timestamp':timestamp, 'for_airline_id':airline_id})
    conn.commit()
    conn.close()
    return all_conversations_list_of_dicts

def get_list_of_conversations_from_file(file_name):#from github use this
    with open(file_name, 'r') as f:
        result = json.load(f)
    return result

In [5]:
replies_trees=get_all_replies_trees_from_csv(trees_file)

In [35]:
all_conversations_list_of_dicts = get_list_of_conversations_from_file(convs_file)

In [37]:
convs_df = pd.DataFrame.from_dict(all_conversations_list_of_dicts)

In [38]:
convs_df

,conv_id,user_id,tweet_id,text,timestamp,for_airline_id
0,0,1662186764,1131173010508062721,Thanks @British_Airways I really needed the ex...,1558527636540,18332190
1,0,18332190,1131175995321520128,@sunriseblade That certainly is annoying. Hope...,1558528348175,18332190
2,0,1662186764,1131180010230951942,"@British_Airways I wish, we're getting off the...",1558529305404,18332190
3,0,18332190,1131316435748360192,@sunriseblade We're pleased to hear the crew d...,1558561831782,18332190
4,0,1662186764,1131319965854965760,@British_Airways BA460,1558562673425,18332190
...,...,...,...,...,...,...
1040225,388307,23849698,1244695647271485448,@VirginAtlantic Booked at one of the Virgin st...,1585593542293,20626359
1040226,388308,38297237,1244692572225896449,I cannot say enough good things about @VirginA...,1585592809145,20626359
1040227,388308,20626359,1244694313407217665,@jenndeavors Hi Jenn. Thats great to hear. Tha...,1585593224275,20626359
1040228,388309,22536055,1244695110639632386,@misubisumisu We've sent you a DM with a message.,1585593414350,22536055
